In [ ]:
##Install Dependencies
!pip install pandas numpy scikit-learn nltk matplotlib seaborn

In [5]:
##push to git
!git status
!git add notebooks/01_data_preprocessing.ipynb
!git commit -m "Added data preprocessing notebook"
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
